# Explore CST-based indexing
## Intro
It's not trivial to *post process* LLM output and then manually piece
together *what exact* `sed` (or similar tool to edit text) command to run.

A possible solution could be to hook up another model to reason about
the action to take (also called an *agent*) to do this for us 😄. However,
let's take a step back and see if feeding additional context works.

Given documents are parsed using
[treesitter](https://tree-sitter.github.io/tree-sitter/)
for additional metadata in the form of a context syntax tree (CST), how
accurate is an answer when this additional metadata is fed in?
For a very simple task it does well:
![Treesitter infused prompt](../assets/treesitter-context-prompt-gpt4-example.png)

Again, there's additional bloat in the response but running the exact command
against the file gives back the desired commitable change:
![git diff after cmd](../assets/diff-after-sed-command.png)

## Goal
Explore treesitter CSTs and add parsed tree output as metadata to a loaded
document.
> 🤔 Note that in previous exploration, chunk size was set to low. Therefore,
a single file probably got split into multiples documents containing a
file/code snippet where a snippet can contain something like
`//Code for @SpringBootApp...` to refer to another related snippet/doc --
keep this in mind!

# Exploration
## Imports

In [1]:
import json
from typing import Any, TypedDict

from dotenv import load_dotenv
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers.language.language_parser import (
    LanguageParser,
)
from langchain_core.documents import Document
from langchain_text_splitters import (
    Language as SplitterLanguage,
)
from rich import print as rprint
from tree_sitter import Language, Node, Parser, Tree

## load env vars

In [2]:
assert load_dotenv(), ".env file with variable present"

## Load & create parser

In [3]:
# load in parser libs for 'properties' file
LANG_NAME = "properties"
JPROP_LANGUAGE = Language("../parsers/ts-properties.so", LANG_NAME)

# create parser
parser = Parser()
parser.set_language(JPROP_LANGUAGE)

/home/bram/.pyenv/versions/3.9.19/envs/calmzeus/lib/python3.9/site-packages/tree_sitter/__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


## Parse file and create CST (context syntax tree)

In [4]:
# mock `application.properties` file
file_contents: bytes = bytes(
    "debug=true\nanotherPropKey='propValue'",
    "utf-8",
)
tree: Tree = parser.parse(file_contents)

### Some info about the tree
#### Root node

In [5]:
rprint(tree.root_node)

<Node type=file, start_point=(0, 0), end_point=(1, 26)>

A node (for the file) spanning from line 0, column 0 to line 1 column 26.

#### tree as a 'string' expression

In [6]:
rprint(tree.root_node.sexp())

(file (property (key) (value)) (property (key) (value)))

#### descendants

In [7]:
print(tree.root_node.descendant_count)

9


## Convert tree to dictionary
Create *pre-order traversal* (first process a node itself, then its children)
algorithm to convert CST to JSON.

In [8]:
class NodeDict(TypedDict):
    grammar_name: str
    text: str
    start: tuple[int, int]
    end: tuple[int, int]
    children: list[Any]  # Ideally of type NodeDict... 🔁


def node_to_dict(node: Node) -> NodeDict:
    node_dict: NodeDict = {
        "grammar_name": node.grammar_name,
        "text": bytes.decode(node.text),
        "start": node.start_point,
        "end": node.end_point,
        "children": [],
    }

    for child in node.children:
        node_dict["children"].append(node_to_dict(child))

    return node_dict

## pretty print JSON of parsed CST

In [9]:
rprint(json.dumps(node_to_dict(tree.root_node), indent=2))

{
  "grammar_name": "file",
  "text": "debug=true\nanotherPropKey='propValue'",
  "start": [
    0,
    0
  ],
  "end": [
    1,
    26
  ],
  "children": [
    {
      "grammar_name": "property",
      "text": "debug=true",
      "start": [
        0,
        0
      ],
      "end": [
        0,
        10
      ],
      "children": [
        {
          "grammar_name": "key",
          "text": "debug",
          "start": [
            0,
            0
          ],
          "end": [
            0,
            5
          ],
          "children": []
        },
        {
          "grammar_name": "=",
          "text": "=",
          "start": [
            0,
            5
          ],
          "end": [
            0,
            6
          ],
          "children": []
        },
        {
          "grammar_name": "value",
          "text": "true",
          "start": [
            0,
            6
          ],
          "end": [
            0,
            10
          ],
          "children": []
        }
      ]
    },
    {
      "grammar_name": "property",
      "text": "anotherPropKey='propValue'",
      "start": [
        1,
        0
      ],
      "end": [
        1,
        26
      ],
      "children": [
        {
          "grammar_name": "key",
          "text": "anotherPropKey",
          "start": [
            1,
            0
          ],
          "end": [
            1,
            14
          ],
          "children": []
        },
        {
          "grammar_name": "=",
          "text": "=",
          "start": [
            1,
            14
          ],
          "end": [
            1,
            15
          ],
          "children": []
        },
        {
          "grammar_name": "value",
          "text": "'propValue'",
          "start": [
            1,
            15
          ],
          "end": [
            1,
            26
          ],
          "children": []
        }
      ]
    }
  ]
}

## Add CST to document metadata
Add *context syntax tree* (CST) dictionary to `.properties` files because
only this parser (language) is set up.
### Load docs

In [10]:
java_code_dir = "/home/bram/projects/heavenlyhades/java/simple-api/"
loader = GenericLoader.from_filesystem(
    java_code_dir,
    glob="**/src/main/**/[!.]*",
    suffixes=[".java", ".properties"],
    parser=LanguageParser(SplitterLanguage.JAVA),
)
documents = loader.load()
print("loaded", len(documents), "docs")

loaded 6 docs


### Enrich metadata with CST

In [11]:
def construct_cst(doc: Document):
    cst = parser.parse(str.encode(doc.page_content))
    return node_to_dict(cst.root_node)


for doc in documents:
    if ".properties" in doc.metadata["source"]:
        doc.metadata["cst"] = construct_cst(doc)

enriched_prop_doc = [
    doc for doc in documents if ".properties" in doc.metadata["source"]
][0]

rprint(enriched_prop_doc)

Document(
    metadata={
        'source': '/home/bram/projects/heavenlyhades/java/simple-api/src/main/resources/application.properties',
        'content_type': 'simplified_code',
        'language': <Language.JAVA: 'java'>,
        'cst': {
            'grammar_name': 'file',
            'text': 'debug=false',
            'start': (0, 0),
            'end': (0, 11),
            'children': [
                {
                    'grammar_name': 'property',
                    'text': 'debug=false',
                    'start': (0, 0),
                    'end': (0, 11),
                    'children': [
                        {'grammar_name': 'key', 'text': 'debug', 'start': (0, 0), 'end': (0, 5), 'children': []},
                        {'grammar_name': '=', 'text': '=', 'start': (0, 5), 'end': (0, 6), 'children': []},
                        {
                            'grammar_name': 'value',
                            'text': 'false',
                            'start': (0, 6),
                            'end': (0, 11),
                            'children': []
                        }
                    ]
                }
            ]
        }
    },
    page_content='debug=false'
)

# Conclusion
Using treesitter to parse Java `.properties` files to get a context tree,
proved pretty straight forward. Note, that for a file *containing a single property*
the CST is already pretty big.

## Improvements
Prevent duplication of file content ('debug=false'); currently duplicated
in file, property and key + value texts. Maybe it's enough to only keep key
and values?